# Active Members Extra Data

In [ ]:
# Import the shared config
import config
import hashlib
import pandas as pd  # type: ignore

# Read raw data
active_members_extra_df = pd.read_csv(config.ACTIVE_MEMBERS_EXTRA_RAW)
# active_members_extra_df.head()

In [ ]:
import pandas as pd
import hashlib  # noqa: F811,


# DataWrangler cleaning code
def clean_data(df):
    # Uppercase column names
    df.columns = df.columns.str.upper()
    # Drop columns with no values
    df = df.dropna(axis=1, how="all")

    # Function to create a short, uppercase hash
    def create_hash(value):
        return hashlib.md5(value.encode()).hexdigest()[:16].upper()

    # Insert AME_PRIMARY_KEY column at the beginning
    df.insert(0, "AME_PRIMARY_KEY", df["MEMBER_NUMBER"].apply(create_hash))
    # Convert to datetime columns that only contain a date
    date_columns = [
        "DATE_JOINED",
        "DATE_EXPIRED",
        "LAST_RENEWAL_DATE",
        "DATE_OF_BIRTH",
        "DATE_MODIFIED",
    ]
    # Use .loc to avoid SettingWithCopyWarning and specify date format
    date_format = "%Y-%m-%d"  # 2024-12-31
    for col in date_columns:
        df.loc[:, col] = pd.to_datetime(df[col], format=date_format, errors="coerce")
    # Convert to datetime columns that cantian both a date and a time
    date_time_format = "%Y-%m-%d %I:%M:%S %p"  # 2024-12-31 11:59:59 PM
    df.loc[:, "LAST_LOGIN_DATETIME"] = pd.to_datetime(
        df["LAST_LOGIN_DATETIME"], format=date_time_format, errors="coerce"
    )

    return df


active_members_extra_clean_df = clean_data(active_members_extra_df.copy())
active_members_extra_clean_df.head()

In [ ]:
# Persist the interim data
active_members_extra_clean_df.to_csv(config.ACTIVE_MEMBERS_EXTRA_INTERIM, index=False)